# Download Resnet models

In [1]:
# !wget "https://d2j0dndfm35trm.cloudfront.net/resnet-18.t7"
# !wget "https://d2j0dndfm35trm.cloudfront.net/resnet-34.t7"
# !wget "https://d2j0dndfm35trm.cloudfront.net/resnet-50.t7"

# Split dataset for validation

In [3]:
import numpy as np
import pandas as pd 

import matplotlib.pyplot as plt
get_ipython().magic(u'matplotlib inline')
plt.rcParams['figure.figsize'] = (14, 14)

In [11]:
labels = pd.read_csv('../data/id_train.csv')
labels_test = pd.read_csv('../data/sample_submission4.csv')
labels_test['label'] = labels.ix[0].label

In [5]:
from sklearn.cross_validation import train_test_split
X_train, X_val =train_test_split(labels, test_size=0.1, stratify = labels['label'],random_state=42)

In [6]:
import os.path
import os
base_path = '../data/data_split/'

!rm -Rf $base_path
for l in X_train.label.unique():
    
    d = base_path + 'train/' + str(l)
    if not os.path.exists(d):
        os.makedirs(d)
        
    d = base_path + 'val/' + str(l)
    if not os.path.exists(d):
        os.makedirs(d)
        

In [7]:
what = X_train 

for i in what.index:
    row = what.ix[i]
    link_name = '../data/data_split/train/' + str(row.label) + '/' +  str(row.Id) + '.jpg'
    if os.path.lexists(link_name):
        os.unlink(link_name)
        
    os.symlink(os.path.abspath('../data/roof_images/' + str(row.Id) + '.jpg'), os.path.abspath(link_name))

In [8]:
what = X_val

for i in what.index:
    row = what.ix[i]
    link_name = '../data/data_split/val/' + str(row.label) + '/' +  str(row.Id) + '.jpg'
    if os.path.lexists(link_name):
        os.unlink(link_name)
        
    os.symlink(os.path.abspath('../data/roof_images/' + str(row.Id) + '.jpg'), os.path.abspath(link_name))

# Prepare for testing

In [9]:
import os.path
import os
base_path = '../data/data_split_test/'

!rm -Rf $base_path
for l in X_train.label.unique():
    
    d = base_path + 'train/' + str(l)
    if not os.path.exists(d):
        os.makedirs(d)
        
    d = base_path + 'val/' + str(l)
    if not os.path.exists(d):
        os.makedirs(d)
        

In [10]:
what = labels 

for i in what.index:
    row = what.ix[i]
    link_name = '../data/data_split_test/train/' + str(row.label) + '/' +  str(row.Id) + '.jpg'
    if os.path.lexists(link_name):
        os.unlink(link_name)
        
    os.symlink(os.path.abspath('../data/roof_images/' + str(row.Id) + '.jpg'), os.path.abspath(link_name))

In [13]:
what = labels_test

for i in what.index:
    row = what.ix[i]
    link_name = '../data/data_split_test/val/' + str(row.label) + '/' +  str(row.Id) + '.jpg'
    if os.path.lexists(link_name):
        os.unlink(link_name)
        
    os.symlink(os.path.abspath('../data/roof_images/' + str(row.Id) + '.jpg'), os.path.abspath(link_name))

# Mix

In [46]:
import torchfile
files = ['preds1.t7','preds2.t7', 'preds3.t7']

class_corresp = torchfile.load('gen/imagenet.t7')['classList']
map_rule = {i:int(x) for i,x in enumerate(class_corresp)}

In [47]:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    return np.exp(x) / np.sum(np.exp(x), axis=1,keepdims=True)

In [48]:
outs =[]
for f in files: 
    p1 = torchfile.load(f)

    names = sum([x[1] for x in p1],[])
    n_idx = np.argsort(names)
    
    outs.append(np.vstack([x[0] for x in p1])[n_idx])

    
names = np.array(names)[n_idx]
ids = [x.split('.')[0].split('/')[-1]  for x in names]

outs = [softmax(x/np.max(x, axis = 1, keepdims=True)) for x in outs]

In [62]:
preds = pd.Series(sum(outs).argmax(1)).map(map_rule)

t = pd.DataFrame()
t['Id'] = ids
t['label'] = preds

# t['image_id'] = t['image_id'].astype(int)
# t = t.sort_values('image_id')
# t[:20]

In [61]:
t.to_csv('resnet50_3av_10crop.csv', index=False)